In [1]:
import pandas as pd
import nltk 
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perfrom cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words

In [2]:
df=pd.read_csv('/Users/swatiraman/Downloads/NLP Hackathon/Hackathon File/TrainData.txt',sep='\t')
df.head(20)

,Answer,Question
0,Yes. It's absolutely beautiful today.,The weather is great isn't it?
1,"yes, i like that one, too.",that one. the one that's all black.
2,it's really nice.,i got it from macy's.
3,at 8:00 p.m.,when does it start?
4,"nothing, except my favorite color is blue.",what's the matter with green eyes?
5,I have read just about everything in Project ...,have you ever read a book
6,a fancy name for applied computer science in ...,what is bioinformatics
7,"Sure, the changing rooms are over there.",Can I try it on?
8,I want it taken from my checking account.,Which account are you making this withdrawal f...
9,and you get to play with a lot of dogs.,that's the truth.


In [3]:
df.shape

(5495, 2)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5495 entries, 0 to 5494
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Answer    5495 non-null   object
 1   Question  5495 non-null   object
dtypes: object(2)
memory usage: 86.0+ KB


In [5]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/swatiraman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [10]:
from gensim.utils import lemmatize

ImportError: cannot import name 'lemmatize' from 'gensim.utils' (/opt/anaconda3/lib/python3.8/site-packages/gensim/utils.py)

In [ ]:
# function that performs text normalization steps

def text_normalization(text):
    text=str(text).lower() # text to lower case
    spl_char_text=re.sub(r'[^ a-z]','',text) # removing special characters
    tokens=nltk.word_tokenize(spl_char_text) # word tokenizing
    lema=wordnet.WordNetLemmatizer() # intializing lemmatization
    tags_list=pos_tag(tokens,tagset=None) # parts of speech
    lema_words=[]   # empty list 
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):  # Verb
            pos_val='v'
        elif pos_token.startswith('J'): # Adjective
            pos_val='a'
        elif pos_token.startswith('R'): # Adverb
            pos_val='r'
        else:
            pos_val='n' # Noun
        lema_token=lema.lemmatize(token,pos_val) # performing lemmatization
        lema_words.append(lema_token) # appending the lemmatized token into a list
    
    return " ".join(lema_words) # returns the lemmatized tokens as a sentence 

In [ ]:
df['lemmatized_text']=df['Question'].apply(text_normalization) # applying the fuction to the dataset to get clean text
df.tail(15)

In [ ]:
# using tf-idf

tfidf=TfidfVectorizer() # intializing tf-id 
x_tfidf=tfidf.fit_transform(df['Question']).toarray() # transforming the data into array


In [ ]:
# returns all the unique word from data with a score of that word

df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names()) 
df_tfidf.head()

In [ ]:
# defining a function that returns response to query using tf-idf

def chat_tfidf(text):
    lemma=text_normalization(text) # calling the function to perform text normalization
    tf=tfidf.transform([lemma]).toarray() # applying tf-idf
    cos=cosine_similarity(df_tfidf,tf) # applying cosine similarity
    index_value=cos.argmax() # getting index value 
    return df['Answer'].loc[index_value]

In [ ]:
test_df = pd.read_csv('/Users/swatiraman/Downloads/NLP Hackathon/Hackathon File/TestData.csv')
test_df.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
answer = []
for text in test_df['Question']:
    #print(text,':',chat_tfidf(text))
    answer.append(chat_tfidf(text))

In [ ]:
test_df['answer']=answer

In [ ]:
#test_df.drop('Answer',axis=1,inplace=True)

In [ ]:
test_df.head(20)

In [ ]:
test_df[['I.D.','answer']]

In [ ]:
test_df[['I.D.','answer']].to_csv('SwatiRaman_3.csv', index=False, encoding='utf-8')

In [ ]:
import pandas as pd
#data = pd.read_csv('qa.csv')
data=pd.read_csv('/Users/swatiraman/Downloads/NLP Hackathon/Hackathon File/TrainData.txt',sep='\t')
# this function is used to get printable results
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])


pip install Levenshtein

In [ ]:
test_data = pd.read_csv('/Users/swatiraman/Downloads/NLP Hackathon/Hackathon File/TestData.csv')
final_data = pd.DataFrame(columns=["Q", "Prediction", "A", "Score"])
from Levenshtein import ratio
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
final_data = getResults(test_data['Question'], getApproximateAnswer)

In [ ]:
final_data.to_csv('final_data.csv', index=False, encoding='utf-8')

In [ ]:
final_data.tail(20)